## Análise exploratória dos dados de queimadas
<hr style="border:2px solid #0077b9;">

- Dados de Queimadas
- Obtendo os dados através da [SimpleGeo.py](https://github.com/vconrado/bdq.py)
- Explorando os dados ([seaborn](https://seaborn.pydata.org/))

# Dados de Queimadas
<hr style="border:2px solid #0077b9;">

<table style="font-size: 26px;">
<thead>
<tr>
<th style="text-align: left">#</th>
<th style="text-align: left">Dado</th>
<th>Descrição</th>
<th style="text-align: center">Formato</th>
<th style="text-align: center">Res. Temp.</th>
<th style="text-align: center">Res. Espacial</th>
<th style="text-align: center">Dimensões</th>
<th style="text-align: center">Tipo</th>
<th style="text-align: center">Serviços</th>
<th style="text-align: center">Tamanho</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align: left">1</td>
<td style="text-align: left">Focos de Queimadas</td>
<td>49.755.288 registros</td>
<td style="text-align: center">PostgreSQL</td>
<td style="text-align: center">Dados obtidos em 2017-08-11</td>
<td style="text-align: center">-</td>
<td style="text-align: center">-</td>
<td style="text-align: center">Feature</td>
<td style="text-align: center">WFS</td>
<td style="text-align: center">5GB+10GB</td>
</tr>
<tr>
<td style="text-align: left">2</td>
<td style="text-align: left">Risco de Fogo</td>
<td>-</td>
<td style="text-align: center">NetCDF</td>
<td style="text-align: center">Diário (366 cenas)</td>
<td style="text-align: center">5km</td>
<td style="text-align: center">1200x1400</td>
<td style="text-align: center">Coverage</td>
<td style="text-align: center">WTSS</td>
<td style="text-align: center">2.3GB</td>
</tr>
<tr>
<td style="text-align: left">3</td>
<td style="text-align: left">Precipitação</td>
<td>-</td>
<td style="text-align: center">NetCDF</td>
<td style="text-align: center">Diário (366 cenas)</td>
<td style="text-align: center">5km</td>
<td style="text-align: center">1200x1400</td>
<td style="text-align: center">Coverage</td>
<td style="text-align: center">WTSS</td>
<td style="text-align: center">2.3GB</td>
</tr>
<tr>
<td style="text-align: left">4</td>
<td style="text-align: left">Temperatura</td>
<td>-</td>
<td style="text-align: center">NetCDF</td>
<td style="text-align: center">Diário (366 cenas)</td>
<td style="text-align: center">5km</td>
<td style="text-align: center">1200x1400</td>
<td style="text-align: center">Coverage</td>
<td style="text-align: center">WTSS</td>
<td style="text-align: center">2.3GB</td>
</tr>
<tr>
<td style="text-align: left">5</td>
<td style="text-align: left">Humidade</td>
<td>-</td>
<td style="text-align: center">NetCDF</td>
<td style="text-align: center">Diário (366 cenas)</td>
<td style="text-align: center">5km</td>
<td style="text-align: center">1200x1400</td>
<td style="text-align: center">Coverage</td>
<td style="text-align: center">WTSS</td>
<td style="text-align: center">2.3GB</td>
</tr>
</tbody>
</table>

<table>
<tr>
<td><img src="./imgs/focos_bra_2016.png" alt="Focos BRA 2016" style="height: 250px;"/></td>
<td><img src="./imgs/risco.png" alt="Risco 2016" style="height: 250px;"/></td>
<td><img src="./imgs/precipitacao.png" alt="Precipitação 2016" style="height: 250px;"/></td>
<td><img src="./imgs/temperatura.png" alt="Temperatura 2016" style="height: 250px;"/></td>
</tr>
<tr>
<td style="text-align: center">1. Focos BRA 2016</td>
<td style="text-align: center">2. Risco 01-01-2016</td>
<td style="text-align: center">3. Precipitação 01-01-2016</td>
<td style="text-align: center">4. Temperatura 01-01-2016</td>
</tr>
<tr>
<td><img src="./imgs/umidade.png" alt="Umidade 2016" style="height: 250px;"/></td>
</tr>
<tr>
<td style="text-align: center">5. Umidade 01-01-2016</td>
</tr>
</table>

# Obtendo dados através da SimpleGeo
<hr style="border:2px solid #0077b9;">

In [ ]:
from SimpleGeo import SimpleGeo, Predicates as pre

import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from IPython.display import display, HTML 
display(HTML("<style>.dataframe{font-size: 16px !important;}</style>"))

# Objeto SimpleGeo/
s = SimpleGeo(wfs="http://www.terrama2.dpi.inpe.br/geoserver", wtss="http://www.terrama2.dpi.inpe.br/e-sensing", cache=True, cache_dir=".simplegeo")

# Coverage e TimeSerie
c = s.coverage("rpth") \
    .attributes(["precipitation", "risk", "temperature", "humidity"])
ts = s.time_serie(c)  

In [ ]:
# Feature Base
f = s.feature('esensing:focos_bra_2016') \
     .attributes(["timestamp","regiao","bioma","satelite","satelite_referencia",
                {'time_serie': ts, 'start_date': 0, 'end_date': 0, 'datetime': 'timestamp'}]) \
     .max_features(5) \
     .sort_by('rand')

# Filtro Base
satelite_filter = pre.OR(
                    pre.EQ('satelite','TERRA_M-M'),
                    pre.EQ('satelite','TERRA_M-T'),
                    pre.EQ('satelite','AQUA_M-M'),
                    pre.EQ('satelite','AQUA_M-T'))

# Obtendo Features para cada Região
fc_n = f.filter(pre.AND(satelite_filter, pre.EQ('regiao','N'))).get()   
fc_ne = f.filter(pre.AND(satelite_filter, pre.EQ('regiao','NE'))).get()   
fc_co = f.filter(pre.AND(satelite_filter, pre.EQ('regiao','CO'))).get()   
fc_se = f.filter(pre.AND(satelite_filter, pre.EQ('regiao','SE'))).get()   
fc_s = f.filter(pre.AND(satelite_filter, pre.EQ('regiao','S'))).get()

In [ ]:
# Combinando todos os dados
focos = pd.concat([fc_n, fc_ne, fc_co, fc_se, fc_s], ignore_index=True)

# Mostrando amostra
display(focos.sample(n=5))

In [ ]:
# Limpando os dados

dados = focos[['humidity', 'precipitation', 'risk', 'temperature', 'bioma', 'regiao', 'timestamp']]

display(dados.sample(n=5))

In [ ]:
# Olhando os dados
dados.describe()

In [ ]:
# Arrumando os dados

# Removendo linhas com missing data
dados = dados[dados['risk']<=1]

# reordenando os dados
dados = dados[["risk", "precipitation", "humidity", "temperature", "bioma", "regiao", "timestamp"]]

# Convertendo temperatura de Kelvin para Celsius
dados["temperature"] = dados["temperature"]-273.15

# Convertendo para data
dados['timestamp'] = pd.to_datetime(dados['timestamp'])  

dados.describe()

## Explorando os dados
<hr style="border:2px solid #0077b9;">
- Explorar dados
    - Todo o Brasil
    - Por região (N, NE, CO, SE e S)
    - Por estação (Primavera, Verão, Outono e Inverno)

### Todo o Brasil
<hr style="border:2px solid #0077b9;">

In [ ]:
# importando e configurando módulos de plotagem
import seaborn as sns
import numpy as np
from IPython.display import display, HTML 
from matplotlib import pyplot as plt
%matplotlib inline 

fig_size = (16,5)
palette=sns.diverging_palette(10, 240,  as_cmap=True)

siglas = ["N ", "NE", "CO", "SE", "S "]
regioes = ["Norte", "Nordeste", "Centro-Oeste", "Sudeste", "Sul"]

palette=sns.diverging_palette(240, 10,  as_cmap=True)

In [ ]:
# Gerando matrix de correlação
sns.set(font_scale=1.2)
f, ax = plt.subplots(figsize=(10, 8))
corr = dados.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=palette, 
            square=True, ax=ax, annot=True) 

In [ ]:
# Boxplot de todos os dados
fig, ax = plt.subplots(figsize=fig_size)
sns.boxplot(ax=ax, x="risk", data=dados, palette="vlag" )
sns.swarmplot(x="risk", data=dados, size=2, color=".3", linewidth=0)
ax.xaxis.grid(True) 
ax.set(ylabel="Todos")
sns.despine(trim=True, left=True)

In [ ]:
# Gerando scatterplot matrix
sns.pairplot(dados)

### Por região
<hr style="border:2px solid #0077b9;">
- N, NE, CO, SE e S

In [ ]:
# Gerando matrix de correlação para cada região

f, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, sharex=True, sharey=True)
ax = [ax1, ax2, ax3, ax4, ax5, ax6]

for i in xrange(0, 5):
    corr = dados[dados['regiao'] == siglas[i]].corr()
    sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=palette, 
                square=True, ax=ax[i], annot=True, vmin=-1, vmax=1)
    ax[i].set_title(regioes[i])

ax6.set_visible(False)
f.subplots_adjust(hspace=0.4, top=4, right=2.5)

In [ ]:
# Gerando scatterplot matrix por região
sns.set(font_scale=2)
color_regions = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]
sns.pairplot(dados, hue="regiao", palette=sns.color_palette(color_regions), size=4)

In [ ]:
# Histogramas para cada região
from scipy import stats, integrate
sns.set(color_codes=True)

sns.set(font_scale=2)
sns.set_context("paper", rc={"font.size":28,"axes.titlesize":22,"axes.labelsize":22})   
f, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, sharex=True, sharey=True)
ax = [ax1, ax2, ax3, ax4, ax5, ax6]

for i in xrange(0, 5):
    sns.distplot(dados[dados['regiao'] == siglas[i]]["risk"], ax=ax[i], kde=False, bins=np.arange(0.0, 1.1, 0.1), color=color_regions[i], hist_kws=dict(edgecolor="k", linewidth=2))
    ax[i].set_title(regioes[i])
    ax[i].tick_params(labelsize=22)
    plt.setp( ax[i].get_xticklabels(), visible=True)

ax[5].set_visible(False)
plt.subplots_adjust(hspace=0.4, top=3, right=1.9)

In [ ]:
# Gerando boxplot para (risco, temperatura, precipitação, umidade) por região

sns.set(font_scale=1.5)

for var in ("risk", "temperature", "precipitation", "humidity"):
    fig, ax = plt.subplots(figsize=fig_size)
    sns.boxplot(ax=ax, x=var,  y="regiao", data=dados, palette=color_regions)
    sns.swarmplot(x=var, y="regiao", data=dados, size=2, color=".3", linewidth=0)
    ax.xaxis.grid(True)
    ax.set(ylabel="")
    sns.despine(trim=True, left=True)

### Por estações
<hr style="border:2px solid #0077b9;">
- Primavera, Verão, Outono e Inverno

In [ ]:
# Separando dados por estação

# períodos das estacoes no hemisfério sul
primavera = (dados['timestamp'] > '2016-09-23') & (dados['timestamp'] <= '2016-12-21')
verao = (dados['timestamp'] > '2016-12-21') | ( (dados['timestamp'] >= '2016-01-01') & (dados['timestamp'] < '2016-03-21') )
outono = (dados['timestamp'] > '2016-03-21') & (dados['timestamp'] <= '2016-06-20')
inverno = (dados['timestamp'] > '2016-06-20') & (dados['timestamp'] <= '2016-09-23')

# Criando coluna estacao
dados['estacao']=''
dados.loc[primavera, 'estacao']="primavera"
dados.loc[verao, 'estacao']="verao"
dados.loc[outono,'estacao']="outono"
dados.loc[inverno, 'estacao']='inverno'

# Contado por grupo de risco e por estação
dados.filter(items=["risk", "estacao"])\
     .groupby(["estacao",pd.cut(dados["risk"], np.arange(0, 1+.25, .25))])\
     .count().rename(columns={'risk':'Total'})

In [ ]:
# Boxplot para Estação-Risco

colors = ["faded green", "windows blue", "amber", "greyish"]
colors_estacoes = sns.xkcd_palette(colors).as_hex()

sns.set(font_scale=1.5)
fig, ax = plt.subplots(figsize=fig_size)
sns.boxplot(ax=ax, x="risk", y="estacao", data=dados, palette=colors_estacoes)
sns.swarmplot(x="risk", y="estacao", data=dados, size=2, color=".3", linewidth=0)
ax.xaxis.grid(True)
ax.set(ylabel="")
sns.despine(trim=True, left=True)

In [ ]:
# Ocorrências por Região-Estação

sns.set(font_scale=1.2)
estacao_regiao=pd.DataFrame(index=("N ", "NE", "CO", "SE", "S "))
for estacao in ("primavera", "verao", "outono", "inverno"): 
    for regiao in ("N ", "NE", "CO", "SE", "S "):
        estacao_regiao.set_value(regiao, estacao, len(dados.loc[ (dados['estacao']==estacao) & (dados['regiao']==regiao)]))

sns.heatmap(estacao_regiao, cmap=palette, annot=True,  fmt='g')

In [ ]:
# Boxplot Região-Estação

sns.set(font_scale=1.5)
for estacao,i in zip(("primavera", "verao", "outono", "inverno"), xrange(0,5)): 
    fig, ax = plt.subplots(figsize=fig_size)
    sns.boxplot(x="risk", y="regiao", data=dados.loc[dados['estacao']==estacao] , palette=color_regions, ax=ax)
    sns.swarmplot(x="risk", y="regiao", data=dados.loc[dados['estacao']==estacao], size=2, color=".3", linewidth=0, ax=ax)
    ax.xaxis.grid(True)
    ax.set(ylabel="")
    sns.despine(trim=True, left=True)
    ax.set_title(estacao)

## Links
<hr style="border:2px solid #0077b9;">


- **simple_geo.py**: [http://github.com/esensing/simple_geo.py/](http://github.com/esensing/simple_geo.py/)
- WTSS.py: [https://github.com/e-sensing/wtss.py](https://github.com/e-sensing/wtss.py)
- Seaborn: [https://seaborn.pydata.org/](https://seaborn.pydata.org/)